In [1]:
import requests
from datetime import datetime, timedelta
import time
import os
import json
import cv2
import urllib.request

app_id = 6747454
app_secured_token = 'SqPOuhy4tven1QcnYCiy'
app_service_token = 'aedca86caedca86caedca86c46aeba5d52aaedcaedca86cf53a3025da08ab36420581a4'

api_version = 5.52

In [2]:
# # get user data like an app

# req_usersget_url = 'https://api.vk.com/method/users.get'
# req_usersget_payload = {'user_id':1, 'v':api_version, 'access_token':app_service_token}
# req_usersget = requests.get(req_usersget_url, params=req_usersget_payload)
# req_usersget.json()

In [3]:
# # get wall data like an app
# req_wallget_domain = 'eternalclassic'

# req_wallget_count = 100
# req_wallget_offset = 0

# req_wallget_url = 'https://api.vk.com/method/wall.get'
# req_wallget_payload = {'domain':req_wallget_domain, 'count':req_wallget_count, 'offset':req_wallget_offset, 'v':api_version, 'access_token':app_service_token}
# req_wallget = requests.get(req_wallget_url, params=req_wallget_payload)
# req_wallget.json()

In [4]:
def get_url_from_attachment(photo):
    photourl = attachment['photo'].get('photo_1280')
    if (photourl is None):
        photourl = attachment['photo'].get('photo_807')
    if (photourl is None):
        photourl = attachment['photo'].get('photo_604')
    if (photourl is None):
        photourl = attachment['photo'].get('photo_130')
    return photourl

In [7]:
# parser

# get user data like an app
# req_wallget_count = 2
# req_wallget_offset = 0

sources = ['mudakoff', 'borsch', 'ovsyanochan', 'ohurmenno', 'maddy_zone', 'fuck_humor', 'dankest_dankest', 'newmemedealer', 'dank_memes_ayylmao', 'ru9gag', 'real_sci_intellectualism', 'phys_kek', 'karkb', '4ch', 'wtf.rasha', 'ru2ch', 'chan4', 'weirdreparametrizationtrick']
N_hours = 120

curdate = datetime.now()
lastdate = curdate - timedelta(hours=N_hours)
# print (curdate, lastdate)


core_dir_path = os.path.join('parsed', str(int(time.time())))
print (core_dir_path)
os.mkdir(core_dir_path)

# timestamp = int(time.mktime(curdate).timetuple())
timestamp = int(time.mktime(curdate.timetuple()))
dataforsave_json = {'date':curdate.strftime('%Y-%m-%d %H:%M:%S'), 'timestamp':timestamp, 'duration':N_hours, 'sources':[], 'data':[]}

counter = 0

for src in sources:
    counter+=1
    ####
    
    req_grouget_url = 'https://api.vk.com/method/groups.getById'
    req_grouget_payload = {'group_id':src, 'fields':['name','members_count'], 'v':api_version, 'access_token':app_service_token}
    req_grouget = requests.get(req_grouget_url, params=req_grouget_payload)
    result = req_grouget.json()
#     print (result)
    
    group = result['response'][0]
    
    name = group['name']
    members_count = group['members_count']
    src_data = {'src':src, 'name':name, 'members_count':members_count}
    dataforsave_json['sources'].append(src_data)
    
    ####
    
    print ('\nParsing ', name, '(',src, ')', ' - ', counter, '/', len(sources),  '...\n')
    stopflag = False
    offset = 0
    cnt = 100
    

    #groups.getById group_id
    
    current_src_dir_path = os.path.join(core_dir_path, src)
    os.mkdir(current_src_dir_path)
    json_raw_current_src_dir_path = os.path.join(current_src_dir_path, 'raw')
    os.mkdir(json_raw_current_src_dir_path)
    images_current_src_dir_path = os.path.join(current_src_dir_path, 'images')
    os.mkdir(images_current_src_dir_path)
#     imagesdata_current_src_dir_path = os.path.join(current_src_dir_path, 'imagesdata')
#     os.mkdir(imagesdata_current_src_dir_path)

    while (not stopflag):
        req_wallget_url = 'https://api.vk.com/method/wall.get'
        req_wallget_payload = {'domain':src, 'count':cnt, 'offset':offset, 'v':api_version, 'access_token':app_service_token}
        req_wallget = requests.get(req_wallget_url, params=req_wallget_payload)
        result = req_wallget.json()
        
        with open(os.path.join(json_raw_current_src_dir_path,str(offset)+'_'+str(cnt)+'.json'), 'w') as outfile_json:
            json.dump(result, outfile_json)
        
        items = result['response']['items']
        for item in items:
            
#             print (item)
            
            is_ads = bool(item.get('marked_as_ads'))
            is_pinned = bool(item.get('is_pinned'))
                        
            if (not is_pinned and not is_ads):
                timestamp = int(item['date'])
                date = datetime.utcfromtimestamp(timestamp)
 
                if (date<lastdate):
                    stopflag = True
                    break 
                
                likes_cnt = item['likes']['count']
                reposts_cnt = item['reposts']['count']
                comments_cnt = item['comments']['count']
                
                attachments = item.get('attachments')
                if (attachments):
                    for attachment in attachments:
                        if (attachment['type']=='photo'):
                            url = get_url_from_attachment(attachment)
                            
                            
#                             minutes, seconds = divmod((curdate-date).total_seconds(), 60)
                            
#                             evaluation = evaluate_meme(likes_cnt,reposts_cnt,comments_cnt)
#                             evaluation_normalised = evaluation/members_count
                            
                            fname = date.strftime('%Y_%m_%d_%H_%M_%S')+'.'+url.split('.')[-1]
                            fname = os.path.join(images_current_src_dir_path,fname)
                            urllib.request.urlretrieve(url, fname)
#                             print (item['id'], src, date, url)
                            
                            curdict_json = dict()
                            curdict_json['id'] = item['id']
                            curdict_json['filepath'] = fname
                            curdict_json['source'] = src
                            curdict_json['likes_cnt'] = likes_cnt
                            curdict_json['reposts_cnt'] = reposts_cnt
                            curdict_json['comments_cnt'] = comments_cnt
                            curdict_json['timestamp'] = timestamp
                            curdict_json['date'] = date.strftime('%Y-%m-%d %H:%M:%S')
#                             curdict_json['evaluation'] = evaluation
#                             curdict_json['evaluation_normalised'] = evaluation_normalised
                            curdict_json['url'] = url
                            dataforsave_json['data'].append(curdict_json)
                            
        offset+=cnt
        
dataforsave_json_path = os.path.join(core_dir_path, 'memes.json')
with open(dataforsave_json_path, 'w') as outfile:  
    json.dump(dataforsave_json, outfile)


                
                
                    
                

parsed/1541888549

Parsing  MDK ( mudakoff )  -  1 / 18 ...



KeyboardInterrupt: 